This notebook contains **10 practical SQL propositions** with a short **business value** explanation. Run each SQL cell in Azure Data Studio using the **SQL** kernel connected to your database.

## Proposition 1: Show the most recent orders (orders with the latest order date).

**Business Value:** Helps a business see the newest orders being processed right now.

In [ ]:
SELECT o.OrderID, o.CustomerID, o.OrderDate
FROM Sales.Orders AS o
WHERE o.OrderDate = (
    SELECT MAX(OrderDate)
    FROM Sales.Orders
);

## Proposition 2: Show customers who never placed an order.

**Business Value:** Identifies inactive customers for follow-ups or re-engagement.

In [ ]:
SELECT c.CustomerID, c.CustomerName
FROM Sales.Customers AS c
WHERE NOT EXISTS (
  SELECT 1
  FROM Sales.Orders AS o
  WHERE o.CustomerID = c.CustomerID
);

## Proposition 3: Show products that were actually sold (appear in order lines).

**Business Value:** Helps focus inventory on items that bring in real sales.

In [ ]:
SELECT si.StockItemID, si.StockItemName
FROM Warehouse.StockItems AS si
WHERE EXISTS (
    SELECT ol.StockItemID
    FROM Sales.OrderLines AS ol
    WHERE ol.StockItemID = si.StockItemID
);

## Proposition 4: Show invoices with more non‑perishable goods than the average invoice.

**Business Value:** Highlights larger‑than‑average orders of non‑perishable goods for inventory planning.

In [ ]:
SELECT o.InvoiceID, o.CustomerID, o.TotalDryItems
FROM Sales.Invoices AS o
WHERE o.TotalDryItems > (
    SELECT AVG(TotalDryItems)
    FROM Sales.Invoices
);

## Proposition 5: Show customers whose credit limit is higher than the average credit limit.

**Business Value:** Highlights customers with higher buying power for special offers or account prioritization.

In [ ]:
SELECT c.CustomerID, c.CustomerName, c.CreditLimit
FROM Sales.Customers AS c
WHERE c.CreditLimit > (
    SELECT AVG(CreditLimit)
    FROM Sales.Customers
);

## Proposition 6: Show the top 5 customers by how many orders they placed.

**Business Value:** Finds the most loyal and active customers to reward or prioritize.

In [ ]:
SELECT TOP (5) dt.CustomerID, c.CustomerName, dt.OrderCount
FROM (
    SELECT o.CustomerID, COUNT(*) AS OrderCount
    FROM Sales.Orders AS o
    GROUP BY o.CustomerID
) AS dt
JOIN Sales.Customers AS c ON c.CustomerID = dt.CustomerID
ORDER BY dt.OrderCount DESC;

## Proposition 7: Show the 10 products that brought in the most money.

**Business Value:** Reveals top‑selling products to guide pricing and marketing.

In [ ]:
SELECT TOP (10)
    si.StockItemID, si.StockItemName, dt.ItemRevenue
FROM (
    SELECT
        ol.StockItemID,
        SUM(ol.Quantity * ol.UnitPrice * (1 + ol.TaxRate/100.0)) AS ItemRevenue
    FROM Sales.OrderLines AS ol
    GROUP BY ol.StockItemID
) AS dt
JOIN Warehouse.StockItems AS si ON si.StockItemID = dt.StockItemID
ORDER BY dt.ItemRevenue DESC;

## Proposition 8: Show 2015 invoices that had more dry items than the average that year.

**Business Value:** Helps spot unusually large non‑perishable shipments for better inventory planning.

In [ ]:
WITH AvgDry AS (
    SELECT AVG(TotalDryItems) AS AvgDryItems
    FROM Sales.Invoices
    WHERE YEAR(InvoiceDate) = 2015
)
SELECT i.InvoiceID, i.CustomerID, i.TotalDryItems
FROM Sales.Invoices AS i, AvgDry
WHERE YEAR(i.InvoiceDate) = 2015
  AND i.TotalDryItems > AvgDry.AvgDryItems;

## Proposition 9: Show how much money each salesperson made in total.

**Business Value:** Lets the business measure sales performance and recognize top employees.

In [ ]:
WITH SalespersonTotals AS (
    SELECT 
        o.SalespersonPersonID,
        SUM(ol.Quantity * ol.UnitPrice * (1 + ol.TaxRate/100.0)) AS Revenue
    FROM Sales.Orders AS o
    JOIN Sales.OrderLines AS ol ON ol.OrderID = o.OrderID
    GROUP BY o.SalespersonPersonID
)
SELECT p.PersonID, p.FullName, st.Revenue
FROM SalespersonTotals AS st
JOIN Application.People AS p ON p.PersonID = st.SalespersonPersonID
ORDER BY st.Revenue DESC;

## Proposition 10: Show customers who placed more than 20 orders.

**Business Value:** Identifies frequent buyers who may qualify for loyalty rewards or discounts.

In [ ]:
WITH OrderCounts AS (
    SELECT CustomerID, COUNT(*) AS OrderCount
    FROM Sales.Orders
    GROUP BY CustomerID
)
SELECT c.CustomerID, c.CustomerName, oc.OrderCount
FROM OrderCounts AS oc
JOIN Sales.Customers AS c ON c.CustomerID = oc.CustomerID
WHERE oc.OrderCount > 20
ORDER BY oc.OrderCount DESC, c.CustomerID;

**Note:** Some queries (4, 7, 9, 10) were refined with the help of an LLM (ChatGPT). Final testing and execution were done manually in DBeaver.